# 99. 자료 출처
## 99-1. 블로그
- 위저원의 블로그 : Build a Steam Game Recomend Model (1/?) - 데이터 수집 경로 설정하기 | [[블로그 링크]](https://weejw.tistory.com/555)

In [1]:
from multiprocessing import Pool, Process
import pandas as pd
import requests
import os, re
import time
import json

In [2]:
def return_or_print(response):
    
    if response.status_code == 200: return response.json()
    else: print(f'no response data with code : {response.status_code}')

    
get_api  = lambda url: return_or_print(requests.get(url))

def get_info(appid):
    
    response = requests.get(f'https://store.steampowered.com/api/appdetails?appids={appid}&l=korean')
    return return_or_print(response)
    
    
def get_reviews(appid):
    
    response = requests.get(f'https://store.steampowered.com/appreviews/{appid}?json=1')
    return return_or_print(response)


def get_requirements(info, platform = 'pc', typeof = 'minimum'):
    
    assert platform in ['pc', 'linux', 'mac'], 'platform must be pc, linux and mac'
    assert typeof in ['minimum', 'recommended'], 'type of must be minimum, recommended'
    
    requirements = info[f'{platform}_requirements'][typeof]
    requirements = re.sub('(<([^>]+)>)', '\n', requirements)
    
    requirements = requirements.split('\n')
    requirements = [req for req in requirements if req != '']
    
    result = {}
    for idx in range(1, len(requirements), 2): 
        
        print(requirements[idx: idx + 2])
        key, value = requirements[idx: idx + 2]
        result[key] = value
        
    return result

In [3]:
steam_games = get_api('https://api.steampowered.com/ISteamApps/GetAppList/v2')['applist']['apps']

In [5]:
appid = steam_games[-3]['appid']
info  = get_info(appid)[str(appid)]['data']

In [6]:
name, typeof      = info['name'], info['type']
appid, is_free    = info['steam_appid'], info['is_free']
short_description = info['short_description']
languages         = info['supported_languages']
image             = info['header_image']
web_site          = info['website']
platforms         = info['platforms']
release_date      = info['release_date']

KeyError: 'supported_languages'

In [ ]:
get_reviews(appid)

In [ ]:
get_info(appid)[str(appid)]